## Setup & EDA

We first need to install all the needed packages

In [1]:
!pip install bitsandbytes
!pip install transformers[torch]
!pip install transformers accelerate
!pip install --upgrade transformers
!pip install datasets


[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: pip install --upgrade pip
zsh:1: no matches found: transformers[torch]

[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


**Restart kernel after installation**

We will then import all the needed libraries

In [2]:
import json
from pathlib import Path
from datasets import Dataset,DatasetDict, load_dataset
import pandas as pd

We reformatted the data set now so that we can use a standard load_dataset

In [3]:
data = load_dataset("json", data_files="train1.json", field="data")

lets just display the dataset to check

In [4]:
data

DatasetDict({
    train: Dataset({
        features: ['translation', 'id'],
        num_rows: 11
    })
})

Great, lest split it up, and create validation and training datasets

In [5]:
split_datasets = data["train"].train_test_split(train_size=0.9, seed=20)
split_datasets["validation"] = split_datasets.pop("test")
split_datasets

DatasetDict({
    train: Dataset({
        features: ['translation', 'id'],
        num_rows: 9
    })
    validation: Dataset({
        features: ['translation', 'id'],
        num_rows: 2
    })
})

In [6]:
split_datasets["train"][1]["translation"]

{'ktl': 'fun main (args: {{type0}} <{{type1}}>)',
 'pli': 'PROCEDURE MAIN {{type0}} {{type1}}'}

## Accelarator

In [7]:
from accelerate import FullyShardedDataParallelPlugin, Accelerator
from torch.distributed.fsdp.fully_sharded_data_parallel import FullOptimStateDictConfig, FullStateDictConfig

fsdp_plugin = FullyShardedDataParallelPlugin(
    state_dict_config=FullStateDictConfig(offload_to_cpu=True, rank0_only=False),
    optim_state_dict_config=FullOptimStateDictConfig(offload_to_cpu=True, rank0_only=False),
)

accelerator = Accelerator(fsdp_plugin=fsdp_plugin)

## Tokenize

Import a tokenizer to convert all our inputs and targets

TOKENs in compiler and LLM/Transformers have different meanings. Tokenisation here refer to the creation of a numerical value for the input tokens, a vector that can be used by the model.

In [8]:
from huggingface_hub import notebook_login

notebook_login()

In [9]:
from transformers import AutoTokenizer

model_checkpoint = "meta-llama/CodeLlama-7b-hf" # Replace this with your desired model
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, return_tensors="pt", use_auth_token=True, force_download=True)


/home/ubuntu/.pyenv/versions/3.10.14/lib/python3.10/site-packages/transformers/models/auto/tokenization_auto.py:757: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(
/home/ubuntu/.pyenv/versions/3.10.14/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/749 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/749 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/411 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/749 [00:00<?, ?B/s]

In [10]:
pip install --upgrade bitsandbytes


[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [11]:
import torch
from transformers import AutoModelForCausalLM, BitsAndBytesConfig

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

model = AutoModelForCausalLM.from_pretrained(model_checkpoint, quantization_config=bnb_config, device_map="auto", force_download=True)

config.json:   0%|          | 0.00/637 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/637 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/home/ubuntu/.pyenv/versions/3.10.14/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

Note because we merged the dataset, we can actually use the same dataset for both source and target.
The most interesting thing here is the tokenizer, a hugging face function, that produces the attention_mask.

In [12]:
pli_sentence = split_datasets["train"][1]["translation"]["pli"]
ktl_sentence = split_datasets["train"][1]["translation"]["ktl"]

inputs = tokenizer(pli_sentence, return_tensors="pt")
targets = tokenizer(ktl_sentence, return_tensors="pt")
inputs, targets

({'input_ids': tensor([[    1, 13756, 29907,  3352, 11499, 14861,  1177,  8620,  1853, 29900,
            930,  8620,  1853, 29896,   930]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])},
 {'input_ids': tensor([[    1,  2090,  1667,   313,  5085, 29901,  8620,  1853, 29900,   930,
            529,  6224,  1853, 29896,   930, 12948]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])})

In [13]:
wrong_targets = tokenizer(ktl_sentence)
print(tokenizer.convert_ids_to_tokens(wrong_targets["input_ids"]))

['<s>', '▁fun', '▁main', '▁(', 'args', ':', '▁{{', 'type', '0', '}}', '▁<', '{{', 'type', '1', '}}', '>)']


A quick function to clean up the input sequence and set the model up to accept the input sequence

overflowing_tokens and num_truncated tokens are things like whitespace, and start of sequence/end of sequence etc.

This model expects the inputs to be named "labels".


In [14]:
max_length = 512

def preprocess_function(examples):
    inputs = [ex["pli"] for ex in examples["translation"]]
    targets = [ex["ktl"] for ex in examples["translation"]]
    
    # Tokenize inputs and targets separately
    model_inputs = tokenizer(inputs, max_length=max_length, truncation=True, padding=True)
    model_targets = tokenizer(targets, max_length=max_length, truncation=True, padding=True)
    
    # Remove unnecessary keys from model_inputs
    model_inputs.pop("overflowing_tokens", None)
    model_inputs.pop("num_truncated_tokens", None)
    
    # Add targets to model_inputs
    model_inputs["labels"] = model_targets["input_ids"]
    
    return model_inputs

In [15]:
tokenizer.pad_token = tokenizer.eos_token

In [16]:
tokenized_datasets = split_datasets.map(
    preprocess_function,
    batched=True,
    remove_columns=split_datasets["train"].column_names,
)

## Set Up LoRA

LoRA, which stands for Low-Rank Adaptation, is a technique introduced to adapt pre-trained models, like those used in natural language processing, with fewer additional parameters. The core idea is to introduce trainable low-rank matrices that adapt the self-attention and feed-forward layers of transformer models. 

In [17]:
!pip install peft

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)



[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [18]:
from peft import prepare_model_for_kbit_training

model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

In [19]:
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )

In [20]:
print(model)

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32016, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=11008, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=11008, bias=False)
          (down_proj): Linear4bit(in_features=11008, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    (norm): Lla

In [21]:
from peft import LoraConfig, get_peft_model

config = LoraConfig(
    r=32,
    lora_alpha=64,
    target_modules=[
        "q_proj",
        "k_proj",
        "v_proj",
        "o_proj",
        "gate_proj",
        "up_proj",
        "down_proj",
        "lm_head",
    ],
    bias="none",
    lora_dropout=0.05,  # Conventional
    task_type="CAUSAL_LM",
)

model = get_peft_model(model, config)
print_trainable_parameters(model)

trainable params: 81109504 || all params: 3581653504 || trainable%: 2.2645826546151575


In [22]:
print(model)

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(32016, 4096)
        (layers): ModuleList(
          (0-31): 32 x LlamaDecoderLayer(
            (self_attn): LlamaSdpaAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=4096, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=32, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=32, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
              )
              (k_proj): lora.Linear4bit(
                (base_layer): Linear4

## Fine-tuning model

In [23]:
if torch.cuda.device_count() > 1: # If more than 1 GPU
    model.is_parallelizable = True
    model.model_parallel = True

In [24]:
from transformers import DataCollatorForSeq2Seq
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [25]:
batch = data_collator([tokenized_datasets["train"][i] for i in range(1, 3)])
batch.keys()

dict_keys(['input_ids', 'attention_mask', 'labels'])

In [31]:
model = accelerator.prepare_model(model)

In [32]:
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments

In [36]:
training_args = Seq2SeqTrainingArguments(
    output_dir="./results",
    evaluation_strategy="no",  # No evaluation during training
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=50,
    predict_with_generate=True,
)

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [37]:
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=None,  # Assuming no evaluation dataset
    data_collator=data_collator,
    tokenizer=tokenizer
)

In [38]:
import os

# Disable wandb
os.environ["WANDB_DISABLED"] = "true"

In [39]:
trainer.train()

/home/ubuntu/.pyenv/versions/3.10.14/lib/python3.10/site-packages/torch/utils/checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss


TrainOutput(global_step=50, training_loss=0.5712627410888672, metrics={'train_runtime': 48.9281, 'train_samples_per_second': 9.197, 'train_steps_per_second': 1.022, 'total_flos': 343121007052800.0, 'train_loss': 0.5712627410888672, 'epoch': 50.0})